In [0]:
import pandas as pd
import logging

In [0]:
# spark.conf.set(
#   "fs.azure.account.key.stge1.dfs.core.windows.net",
#   "access key"
# )    commented bcz git is restricting the key

In [0]:
bronze_log_path = "abfss://bronze@stge1.dfs.core.windows.net/logs/"

In [0]:
import datetime
from pyspark.sql import SparkSession
bronze_log_path = "abfss://bronze@stge1.dfs.core.windows.net/logs/"
log_data = []
current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [0]:
try: 
    df1 = spark.read.parquet("abfss://silver@stge1.dfs.core.windows.net/team_silver.parquet")
    df2 = spark.read.parquet("abfss://silver@stge1.dfs.core.windows.net/Player__Performance_silver.parquet")
    df3 = spark.read.parquet("abfss://silver@stge1.dfs.core.windows.net/stadium_silver.parquet")
    df4 = spark.read.parquet("abfss://silver@stge1.dfs.core.windows.net/player_team_silver.parquet")
    df5 = spark.read.parquet("abfss://silver@stge1.dfs.core.windows.net/match_stadium_silver.parquet")
    df6 = spark.read.parquet("abfss://silver@stge1.dfs.core.windows.net/match_performance_silver.parquet")
    log_data.append((current_time, "INFO", "File found"))

except:
    print("Error: File not found")
log_data.append((current_time, "ERROR", "File not found"))


In [0]:
try:
    df2 = df2.withColumnRenamed("runs_scored", "runs_scored_match")
    df2 = df2.withColumnRenamed("wickets_taken", "wickets_taken_match")
    df2.display()
    log_data.append((current_time, "INFO", "File found"))
except:
    print("Error: File not found")
log_data.append((current_time, "ERROR", "File not found"))

match_id,player_id,runs_scored_match,wickets_taken_match,ball_taken
214,101,82,2,50
211,102,97,2,50
207,103,29,2,50
218,104,14,3,50
209,105,5,1,50
218,106,44,1,50
202,107,68,2,50
213,108,94,4,50
215,109,93,2,50
219,110,75,3,50


In [0]:
from pyspark.sql.functions import current_date, input_file_name,regexp_extract
try:
    df1 = df1.withColumn("ingestion_date", current_date()).withColumn("source_file", regexp_extract(input_file_name(), r"([^/]+$)", 1))
    df2 = df2.withColumn("ingestion_date", current_date()).withColumn("source_file", regexp_extract(input_file_name(), r"([^/]+$)", 1))
    df3 = df3.withColumn("ingestion_date", current_date()).withColumn("source_file", regexp_extract(input_file_name(), r"([^/]+$)", 1))
    df4 = df4.withColumn("ingestion_date", current_date()).withColumn("source_file", regexp_extract(input_file_name(), r"([^/]+$)", 1))
    df5 = df5.withColumn("ingestion_date", current_date()).withColumn("source_file", regexp_extract(input_file_name(), r"([^/]+$)", 1))
    df6 = df6.withColumn("ingestion_date", current_date()).withColumn("source_file", regexp_extract(input_file_name(), r"([^/]+$)", 1))
    log_data.append((current_time, "INFO", "File found"))
except:
    print("Error: File not found")
    log_data.append((current_time, "ERROR", "File not found"))


In [0]:
# checking datatypes of each column

df1.printSchema()
df2.printSchema()
df3.printSchema()
df4.printSchema()
df5.printSchema()
df6.printSchema()

root
 |-- team_id: integer (nullable = true)
 |-- team_name: string (nullable = true)
 |-- home_ground: string (nullable = true)
 |-- captain: string (nullable = true)
 |-- ingestion_date: date (nullable = false)
 |-- source_file: string (nullable = false)

root
 |-- match_id: integer (nullable = true)
 |-- player_id: integer (nullable = true)
 |-- runs_scored_match: integer (nullable = true)
 |-- wickets_taken_match: integer (nullable = true)
 |-- ball_taken: integer (nullable = true)
 |-- ingestion_date: date (nullable = false)
 |-- source_file: string (nullable = false)

root
 |-- stadium_id: integer (nullable = true)
 |-- stadium_name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- capacity: integer (nullable = true)
 |-- ingestion_date: date (nullable = false)
 |-- source_file: string (nullable = false)

root
 |-- player_id: integer (nullable = true)
 |-- player_name: string (nullable = true)
 |-- team_id: integer (nullable = true)
 |-- player_role: string (null

In [0]:
# changing data type of runs_scored column from double to integer

df6=df6.withColumn('runs_scored', df6['runs_scored'].cast('integer'))
df6.printSchema()

root
 |-- match_id: integer (nullable = true)
 |-- team_id: integer (nullable = true)
 |-- opponent_team_id: integer (nullable = true)
 |-- runs_scored: integer (nullable = true)
 |-- wickets_taken: integer (nullable = true)
 |-- match_result: string (nullable = true)
 |-- ingestion_date: date (nullable = false)
 |-- source_file: string (nullable = false)



In [0]:
try: 
    df1_slv=df1.dropna()
    df2_slv=df2.dropna()
    df3_slv=df3.dropna()
    df4_slv=df4.dropna()
    df5_slv=df5.dropna()
    df6_slv=df6.dropna()
    log_data.append((current_time, "INFO", "File found"))
except:
    print("Error: File not found")
    log_data.append((current_time, "ERROR", "File not found"))


In [0]:
try:
    dm1=df4_slv.join(df1_slv, df4_slv.team_id==df1_slv.team_id, how='inner').drop(df1_slv.team_id,df1_slv.ingestion_date,df4_slv.ingestion_date,df1_slv.source_file,df4_slv.source_file)
    log_data.append((current_time, "INFO", "File found"))
except:
    print("Error: File not found")


In [0]:
try:
    dm2=df6_slv.join(dm1, df6_slv.team_id==dm1.team_id, how='inner').drop(df6_slv.team_id,df6_slv.ingestion_date,df6_slv.source_file)
    log_data.append((current_time, "INFO", "File found"))
except:
    print("Error: File not found")

In [0]:
try:
    dm3=df5_slv.join(dm2, df5_slv.match_id==dm2.match_id, how='inner').drop(df5_slv.match_id,df5_slv.ingestion_date,df5_slv.source_file)
    log_data.append((current_time, "INFO", "File found"))
except:
    print("Error: File not found")

In [0]:
try:
    dm4=df3_slv.join(dm3, df3_slv.stadium_id==dm3.stadium_id, how='inner').drop(df3_slv.ingestion_date,df3_slv.source_file,df3_slv.stadium_id)
    log_data.append((current_time, "INFO", "File found"))
except:
    print("Error: File not found")

In [0]:
try:    
    joined_df=df2_slv.join(dm4, df2_slv.player_id==dm4.player_id, how='inner').drop(df2_slv.player_id,df2_slv.match_id,df2_slv.ingestion_date,df2_slv.source_file)
    joined_df.display()
    log_data.append((current_time, "INFO", "File found"))
except:
    print("Error: File not found")

runs_scored_match,wickets_taken_match,ball_taken,stadium_name,city,capacity,stadium_id,match_id,opponent_team_id,runs_scored,wickets_taken,match_result,player_id,player_name,team_id,player_role,team_name,home_ground,captain
82,2,50,Arun Jaitley Stadium,Delhi,41820,303,212,5,227,6,Win,101,Thomas Mendez,1,All-rounder,Mumbai Indians,Wankhede Stadium,Rohit Sharma
82,2,50,Eden Gardens,Kolkata,66000,304,202,3,202,10,Loss,101,Thomas Mendez,1,All-rounder,Mumbai Indians,Wankhede Stadium,Rohit Sharma
14,3,50,Arun Jaitley Stadium,Delhi,41820,303,219,3,194,7,Loss,104,Amber Bell,2,Bowler,Chennai Super Kings,M. A. Chidambaram Stadium,MS Dhoni
14,3,50,Eden Gardens,Kolkata,66000,304,218,1,190,5,Tie,104,Amber Bell,2,Bowler,Chennai Super Kings,M. A. Chidambaram Stadium,MS Dhoni
5,1,50,Arun Jaitley Stadium,Delhi,41820,303,212,5,227,6,Win,105,Shawn Bowers,1,Batsman,Mumbai Indians,Wankhede Stadium,Rohit Sharma
5,1,50,Eden Gardens,Kolkata,66000,304,202,3,202,10,Loss,105,Shawn Bowers,1,Batsman,Mumbai Indians,Wankhede Stadium,Rohit Sharma
94,4,50,Arun Jaitley Stadium,Delhi,41820,303,219,3,194,7,Loss,108,Jared Knox,2,Bowler,Chennai Super Kings,M. A. Chidambaram Stadium,MS Dhoni
94,4,50,Eden Gardens,Kolkata,66000,304,218,1,190,5,Tie,108,Jared Knox,2,Bowler,Chennai Super Kings,M. A. Chidambaram Stadium,MS Dhoni
93,2,50,Arun Jaitley Stadium,Delhi,41820,303,219,3,194,7,Loss,109,Kevin Conrad,2,Bowler,Chennai Super Kings,M. A. Chidambaram Stadium,MS Dhoni
93,2,50,Eden Gardens,Kolkata,66000,304,218,1,190,5,Tie,109,Kevin Conrad,2,Bowler,Chennai Super Kings,M. A. Chidambaram Stadium,MS Dhoni


In [0]:
# storing joined table data in silver cointainer as parquet file 
try:
    joined_df.write.mode("overwrite").parquet("abfss://silver@stge1.dfs.core.windows.net/joined_df_silver.parquet")
    log_data.append((current_time, "INFO", "File found"))
except:
    print("Error: File not found")

In [0]:
try:
    jdbc_url = "jdbc:sqlserver://pojectserver.database.windows.net:1433;databaseName=proj_db;encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;"


    connection_properties = {
        "user": "shree",
        "password": "proj_data@123",
        "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
    }
    log_data.append((current_time, "INFO", "Connection Succesful"))
except:
    print("Error: Connection Failed")

In [0]:
try:  
    joined_df.write \
        .format("jdbc") \
        .option("url", jdbc_url) \
        .option("dbtable", "silver_db.final_silver") \
        .option("user", "shree") \
        .option("password", "proj_data@123") \
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
        .mode("overwrite") \
        .save()
    log_data.append((current_time, "INFO", "File saved"))
except:
    print("Error: File not saved")